## PGA Draftkings Notebook
Use [PGA Website](https://www.pgatour.com/tournaments/schedule.html) to look up tournament info and fill out first USER INPUT block below. (Fetch/XHR from Network tab, Paylod sub-tab)

Looks like the tournament ID is also in the address bar.

### User Input

In [1]:
# === USER INPUTS ===
# Old Tournament
old_tournament_name = "World Wide Technology Championship"
tournament_date = "11/9/2025"  # Ending date of tournament
old_course = "El Cardonal at Diamante"
tournament_id = "R2025457"  # Tournament ID from the PGA Tour API - Also visible in the URL of the tournament page

# New Tournament
new_tournament_name = "Butterfield Bermuda Championship"
new_ending_date = "11/16/2025"
new_course = "Port Royal Golf Course"
new_season = 2025


# === LIBRARIES AND VARIABLES ===
# Import necessary libraries
import requests
import pandas as pd
from datetime import datetime
import sqlite3 as sql
import numpy as np
from numpy import nan
import os
import importlib
import utils.db_utils
from utils.db_utils import TOURNAMENT_NAME_MAP, PLAYER_NAME_MAP

# === TOURNAMENT CONFIG ===
tournament_config = {
    "old": {
        "name": old_tournament_name,
        "date": tournament_date,
        "course": old_course,
        "id": tournament_id
    },
    "new": {
        "name": new_tournament_name,
        "course": new_course,
        "season": new_season,
        "ending_date": pd.to_datetime(new_ending_date, format="%m/%d/%Y"),
        "quoted_course": f'"{new_course}"',
        "quoted_name": f"'{new_tournament_name}'"
    }
}

this_week_key = str(tournament_config["new"]["ending_date"].date())

# Create a minimal one-row history DataFrame
this_week_history = pd.DataFrame([{
    "SEASON": tournament_config["new"]["season"],
    "TOURNAMENT": tournament_config["new"]["name"],
    "ENDING_DATE": tournament_config["new"]["ending_date"],
    "COURSE": tournament_config["new"]["course"],
    "TOURN_ID": None
}])

### Update Database

#### Old Tournament

In [2]:
importlib.reload(utils.db_utils)  # Only needed if you're actively editing db_utils.py
from utils.db_utils import update_tournament_results

# Change these each year!!
season = 2025
year = 20250  # Unique GraphQL year distinguishing number in case of multiple per year

# Run the update
db_path = "data/golf.db"  # Or use os.path.join("data", "golf.db")
tournDf = update_tournament_results(tournament_config, db_path, season, year)

# Show just the most recent tournament added for confirmation
from sqlalchemy import create_engine

engine = create_engine(f"sqlite:///{db_path}")

query = f"""
SELECT *
FROM tournaments
WHERE TOURN_ID = '{tournament_config['old']['id']}'
  AND ENDING_DATE = '{datetime.strptime(tournament_config['old']['date'], '%m/%d/%Y').date()}'
"""

recent = pd.read_sql(query, engine)
engine.dispose()
recent.sort_values(by='FINAL_POS').head()

📦 Fetching results for tournament ID R2025457 (World Wide Technology Championship), year: 20250
✅ 120 new rows added for 'World Wide Technology Championship'


,SEASON,ENDING_DATE,TOURN_ID,TOURNAMENT,COURSE,PLAYER,POS,FINAL_POS,ROUNDS:1,ROUNDS:2,ROUNDS:3,ROUNDS:4,OFFICIAL_MONEY,FEDEX_CUP_POINTS
0,2025,2025-11-09,R2025457,World Wide Technology Championship,El Cardonal at Diamante,Ben Griffin,1,1,-7,-7,-6,-9,"$1,080,000.00",0.000
1,2025,2025-11-09,R2025457,World Wide Technology Championship,El Cardonal at Diamante,Chad Ramey,T2,2,-8,-7,-5,-7,"$534,000.00",245.000
2,2025,2025-11-09,R2025457,World Wide Technology Championship,El Cardonal at Diamante,Sami Valimaki,T2,2,-11,-5,-3,-8,"$534,000.00",245.000
3,2025,2025-11-09,R2025457,World Wide Technology Championship,El Cardonal at Diamante,Garrick Higgo,T4,4,-4,-7,-11,-4,"$270,000.00",122.500
4,2025,2025-11-09,R2025457,World Wide Technology Championship,El Cardonal at Diamante,Trevor Cone,T4,4,-5,-8,-7,-6,"$270,000.00",122.500


#### Stats

In [19]:
importlib.reload(utils.db_utils)
from utils.db_utils import update_season_stats  # <- This line is essential

# Change these each year!!
statsYear = 2025

stats_df = update_season_stats(statsYear, db_path)
stats_df.head()

✅ Overwrote stats for season 2025 with 999 rows.


,PLAYER,SGTTG_RANK,SGTTG,SGOTT_RANK,SGOTT,SGAPR_RANK,SGAPR,SGATG_RANK,SGATG,SGP_RANK,SGP,BIRDIES_RANK,BIRDIES,PAR_3_RANK,PAR_3,PAR_4_RANK,PAR_4,PAR_5_RANK,PAR_5,TOTAL_DRIVING_RANK,TOTAL_DRIVING,DRIVING_DISTANCE_RANK,DRIVING_DISTANCE,DRIVING_ACCURACY_RANK,DRIVING_ACCURACY,GIR_RANK,GIR,SCRAMBLING_RANK,SCRAMBLING,OWGR_RANK,OWGR,SEASON
0,A.J. Ewart,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,483.0,.3035,2025
1,Aaron Baddeley,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,682.0,.1848,2025
2,Aaron Cockerill,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,417.0,.3684,2025
3,Aaron Rai,11.0,0.942,22.0,0.391,22.0,0.476,67.0,0.077,128.0,-0.134,99.0,21.99%,176.0,3.16,13.0,3.97,29.0,4.54,74.0,172,170.0,289.6,2.0,73.85%,16.0,70.52%,95.0,59.16%,23.0,3.2384,2025
4,Aaron Wilkin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,993.0,.0929,2025


#### Odds
Not usually needed for weekly routine.

**Manual Fix! Odds name cleanup (only needed when joins fail)**

Make sure to update the dictionaries in db_utils.py if new names need to be added.

In [17]:
importlib.reload(utils.db_utils)
from utils.db_utils import clean_odds_names, PLAYER_NAME_MAP, TOURNAMENT_NAME_MAP

db_path = "data/golf.db" 
updated_odds = clean_odds_names(db_path, TOURNAMENT_NAME_MAP, PLAYER_NAME_MAP)
updated_odds.head()

✅ Cleaned and updated 516 rows in 'odds' table.


,SEASON,TOURNAMENT,ENDING_DATE,PLAYER,ODDS,VEGAS_ODDS,TOURNAMENT_ORIG,PLAYER_ORIG
8275,2024,World Wide Technology Championship,2024-11-10,Max Greyserman,16/1,16.0,World Wide Tech Champ,Max Greyserman
8276,2024,World Wide Technology Championship,2024-11-10,Doug Ghim,18/1,18.0,World Wide Tech Champ,Doug Ghim
8277,2024,World Wide Technology Championship,2024-11-10,J.J. Spaun,25/1,25.0,World Wide Tech Champ,J.J. Spaun
8278,2024,World Wide Technology Championship,2024-11-10,Beau Hossler,25/1,25.0,World Wide Tech Champ,Beau Hossler
8279,2024,World Wide Technology Championship,2024-11-10,Ben Griffin,25/1,25.0,World Wide Tech Champ,Ben Griffin


**Historical Odds Updates**

Only run this when loading in entire year odds at the start of each year or if corrections need to be made.  This will load in the entire year into the database and update it using the dictionary in db_utils.py.

In [15]:
importlib.reload(utils.db_utils)
from utils.db_utils import import_historical_odds

oddsYear = "2022-2023"    # URL segment
season = 2023             # PGA Tour season
db_path = "data/golf.db"

odds_df = import_historical_odds(oddsYear, season, db_path)
odds_df.head()

✅ Inserting 120 new rows into odds table...


,SEASON,TOURNAMENT,ENDING_DATE,PLAYER,ODDS,VEGAS_ODDS
4,2023,Fortinet Championship,2022-09-18,Hideki Matsuyama,18/1,18.0
5,2023,Fortinet Championship,2022-09-18,Max Homa,18/1,18.0
6,2023,Fortinet Championship,2022-09-18,Corey Conners,20/1,20.0
7,2023,Fortinet Championship,2022-09-18,Maverick McNealy,25/1,25.0
8,2023,Fortinet Championship,2022-09-18,Taylor Pendrith,25/1,25.0


**Not normally needed**

The code below is a way to troubleshoot the odds function that is in db_utils.py.  If it does not pull the stats correctly, we can run it outside of the db_utils.py file and explore what might be wrong with it, iterating until it works, and then using that fix to update db_utils.py.  This is not normally needed, but wanted to leave this in place as a reference because this odds website can be tricky.

In [16]:
import pandas as pd
import numpy as np
import requests
import re
from datetime import datetime
from io import StringIO

# === USER INPUT ===
oddsYear = "2020-2021"    # URL segment
season = 2021        # PGA Tour season

url = f"http://golfodds.com/archives-{oddsYear}.html"
response = requests.get(url)
tables = pd.read_html(StringIO(response.text))
# raw_df = tables[5]  # the actual table of interest
# Find the largest 2-column table that contains at least some odds-like strings
raw_df = None
for tbl in tables:
    if tbl.shape[1] == 2 and tbl.shape[0] > 50:  # Rough filter
        sample = tbl.iloc[:, 1].astype(str).str.contains(r"\d+/\d+").sum()
        if sample > 5:
            raw_df = tbl
            break

if raw_df is None:
    raise ValueError("❌ Could not find valid odds table on the page.")

# === STEP 1: Initial clean-up ===
df = raw_df.dropna(how="all").reset_index(drop=True)
df.columns = ["PLAYER", "ODDS"]

# 🔧 Clean up non-breaking spaces and extra whitespace
df["PLAYER"] = (
    df["PLAYER"]
    .astype(str)
    .str.replace("\xa0", " ", regex=False)
    .str.replace(r"\s+", " ", regex=True)
    .str.strip()
)

df.insert(loc=0, column="SEASON", value=season)
df.insert(loc=1, column="TOURNAMENT", value=np.nan)
df.insert(loc=2, column="ENDING_DATE", value=np.nan)

# === STEP 2: Helper function for parsing date strings ===
def parse_ending_date(text):
    import re
    from datetime import datetime

    # Normalize whitespace and symbols
    text = (
        text.replace("\u2013", "-")
            .replace("–", "-")
            .replace("\xa0", " ")
    )
    text = re.sub(r"\bSept(?!ember)\b", "Sep", text)

    # ✅ Fix typo: "Match" → "March" only when used in a date context
    text = re.sub(r"\bMatch(?=\s+\d{1,2}\s*[-–]\s*\d{1,2},\s*\d{4})", "March", text)

    # Pattern 1: "July 30 - August 2, 2015" or "Oct 29 - Nov 1, 2015"
    match = re.search(r"(\w+)\s\d+\s*-\s*(\w+)\s(\d+),\s(\d{4})", text)
    if match:
        month2, day2, year = match.group(2), match.group(3), match.group(4)
        for fmt in ["%B %d, %Y", "%b %d, %Y"]:
            try:
                return datetime.strptime(f"{month2} {day2}, {year}", fmt).date()
            except ValueError:
                continue

    # Pattern 2: "November 21-24, 2024"
    match = re.search(r"(\w+)\s\d+-\d+,\s(\d{4})", text)
    if match:
        month, year = match.group(1), match.group(2)
        day = re.search(r"(\d+)-(\d+)", text).group(2)
        for fmt in ["%B %d, %Y", "%b %d, %Y"]:
            try:
                return datetime.strptime(f"{month} {day}, {year}", fmt).date()
            except ValueError:
                continue

    # Pattern 3: "Sunday, October 20, 2019"
    try:
        return datetime.strptime(text.strip(), "%A, %B %d, %Y").date()
    except ValueError:
        pass

    # Pattern 4: "October 20, 2019"
    try:
        return datetime.strptime(text.strip(), "%B %d, %Y").date()
    except ValueError:
        pass

    return None

last_tourn_name = None
last_date = None

# === STEP 3: Iterate block by block ===
final_rows = []
i = 0
last_tourn_name = None
last_end_date = None

while i < len(df) - 4:
    player_i = str(df.loc[i, "PLAYER"])
    player_i2 = str(df.loc[i + 2, "PLAYER"])
    player_i3 = str(df.loc[i + 3, "PLAYER"]).lower()

    # Detect start of a new tournament block
    is_header = (
        pd.isna(df.loc[i, "ODDS"]) and
        pd.isna(df.loc[i + 1, "ODDS"]) and (
            re.search(r"\w+\s\d+\s*[-–]\s*(\w+\s)?\d+,\s\d{4}", player_i2) or
            re.search(r"(Monday|Tuesday|Wednesday|Thursday|Friday|Saturday|Sunday),?\s+\w+\s\d{1,2},\s\d{4}", player_i2)
        )
    )

    if is_header:
        tourn_name = player_i.strip()
        end_date = parse_ending_date(player_i2)

        # Skip cancelled or empty blocks
        if "cancelled" in player_i3:
            print(f"⚠️ Skipping cancelled tournament: {tourn_name} — {end_date}")
            i += 4
            continue

        # Avoid duplicate block processing
        if tourn_name == last_tourn_name and end_date == last_end_date:
            i += 1
            continue

        print(f"📍 Detected: {tourn_name} — Ending: {end_date}")
        last_tourn_name = tourn_name
        last_end_date = end_date
        i += 4  # Skip header lines

        # Collect all player rows until next header block
        while i < len(df) - 2:
            next_i2 = str(df.loc[i + 2, "PLAYER"])
            is_next_header = (
                pd.isna(df.loc[i, "ODDS"]) and
                pd.isna(df.loc[i + 1, "ODDS"]) and (
                    re.search(r"\w+\s\d+\s*[-–]\s*(\w+\s)?\d+,\s\d{4}", next_i2) or
                    re.search(r"(Monday|Tuesday|Wednesday|Thursday|Friday|Saturday|Sunday),?\s+\w+\s\d{1,2},\s\d{4}", next_i2)
                )
            )
            if is_next_header:
                break

            if pd.notna(df.loc[i, "ODDS"]):
                row = df.loc[i].copy()
                row["TOURNAMENT"] = tourn_name
                row["ENDING_DATE"] = end_date
                final_rows.append(row)
            i += 1
    else:
        i += 1

# === STEP 4: Create cleaned DataFrame ===
clean_df = pd.DataFrame(final_rows)

# ✅ Prevent crash if nothing was parsed
if clean_df.empty or "PLAYER" not in clean_df.columns:
    print(f"⚠️ No valid tournament blocks detected for season {season} ({oddsYear})")
    final_df = pd.DataFrame()  # Safe fallback
else:
    # Remove winner tag
    clean_df["PLAYER"] = clean_df["PLAYER"].str.replace(r"\s\*Winner\*", "", regex=True)

    # Clean odds to numeric
    clean_df["VEGAS_ODDS"] = (
        clean_df["ODDS"]
        .str.replace(",", "")
        .str.extract(r"(\d+)/(\d+)")
        .astype(float)
        .apply(lambda x: x[0] / x[1], axis=1)
    )

    # Final output with source index for debugging
    final_df = clean_df[
        ["SEASON", "TOURNAMENT", "ENDING_DATE", "PLAYER", "ODDS", "VEGAS_ODDS"]
    ].reset_index(drop=True)

    # Drop non-standard team events (e.g., Presidents Cup, Ryder Cup)
    drop_terms = ["Presidents Cup", "Ryder Cup"]
    final_df = final_df[~final_df["TOURNAMENT"].str.contains("|".join(drop_terms), case=False, na=False)]

    display(final_df.head())



📍 Detected: Safeway Open — Ending: 2020-09-13
📍 Detected: US Open — Ending: 2020-09-20
📍 Detected: R & C Championship — Ending: 2020-09-27
📍 Detected: at Big Cedar Lodge - — Ending: 2020-09-22
📍 Detected: Sanderson Farms Champ — Ending: 2020-10-04
📍 Detected: Shriners H for C Open — Ending: 2020-10-11
📍 Detected: The CJ Cup — Ending: 2020-10-18
📍 Detected: ZOZO CHAMPIONSHIP — Ending: 2020-10-25
📍 Detected: Bermuda Championship — Ending: 2020-11-01
📍 Detected: Vivint Houston Open — Ending: 2020-11-08
📍 Detected: The Masters — Ending: 2020-11-15
📍 Detected: The RSM Classic — Ending: 2020-11-22
📍 Detected: Champions for Change — Ending: 2020-11-27
📍 Detected: Mayakoba Golf Classic — Ending: 2020-12-06
📍 Detected: QBE Shootout — Ending: 2020-12-13
📍 Detected: Sentry Tourn of Champions — Ending: 2021-01-10
📍 Detected: Sony Open in Hawaii — Ending: 2021-01-17
📍 Detected: The American Express — Ending: 2021-01-24
📍 Detected: Abu Dhabi HSBC Champ — Ending: 2021-01-24
📍 Detected: Farmers Insura

,SEASON,TOURNAMENT,ENDING_DATE,PLAYER,ODDS,VEGAS_ODDS
0,2021,Safeway Open,2020-09-13,Phil Mickelson,20/1,20.0
1,2021,Safeway Open,2020-09-13,Si Woo Kim,20/1,20.0
2,2021,Safeway Open,2020-09-13,Brendan Steele,20/1,20.0
3,2021,Safeway Open,2020-09-13,Shane Lowry,25/1,25.0
4,2021,Safeway Open,2020-09-13,Sergio Garcia,30/1,30.0


In [17]:
from datetime import datetime, date
# ✅ Check for non-date types in ENDING_DATE
non_dates = final_df[~final_df["ENDING_DATE"].apply(lambda x: isinstance(x, date))]

print(f"🧪 Rows with invalid ENDING_DATE values: {len(non_dates)}")
display(non_dates.head(10))



🧪 Rows with invalid ENDING_DATE values: 0


,SEASON,TOURNAMENT,ENDING_DATE,PLAYER,ODDS,VEGAS_ODDS


In [18]:
dupes = final_df.duplicated(subset=["SEASON", "TOURNAMENT", "ENDING_DATE", "PLAYER"], keep=False)

print(f"🚨 Duplicate primary keys in final_df: {dupes.sum()}")
display(final_df[dupes].sort_values(by=["SEASON", "TOURNAMENT", "PLAYER"]))

🚨 Duplicate primary keys in final_df: 0


,SEASON,TOURNAMENT,ENDING_DATE,PLAYER,ODDS,VEGAS_ODDS


#### Misc. Cleaning
Not normally needed

In [26]:
importlib.reload(utils.db_utils)
from utils.db_utils import clean_player_names_in_table, PLAYER_NAME_MAP

db_path = "data/golf.db"

# Run for all relevant tables
clean_player_names_in_table(db_path, "tournaments", PLAYER_NAME_MAP)
clean_player_names_in_table(db_path, "stats", PLAYER_NAME_MAP)
clean_player_names_in_table(db_path, "odds", PLAYER_NAME_MAP)

ℹ️ No player names needed updates in 'tournaments'.
ℹ️ No player names needed updates in 'stats'.
ℹ️ No player names needed updates in 'odds'.


,SEASON,TOURNAMENT,ENDING_DATE,PLAYER,ODDS,VEGAS_ODDS,PLAYER_ORIG


## Historical Data

### Pull Relevant Seasons
Do a check to see when this course or tournament have been historically played.

In [2]:
importlib.reload(utils.db_utils)
from utils.db_utils import get_combined_history_seasons

# === USER INPUT ===
seasons = list(range(2016, 2025))  # Adjust as needed
db_path = "data/golf.db"

# Pull course and tournament from config
n_course = tournament_config["new"]["course"]
n_tourn = tournament_config["new"]["name"]

# Fetch relevant history
history_df = get_combined_history_seasons(db_path, course=n_course, tournament=n_tourn, allowed_seasons=seasons)
history_df.head(20)


ℹ️ Found 5 relevant tournaments from course or tournament name.


,SEASON,COURSE,TOURN_ID,TOURNAMENT,ENDING_DATE
258,2020,Port Royal Golf Course,528,Bermuda Championship,2019-11-03
126,2021,Port Royal Golf Course,528,Bermuda Championship,2020-11-01
0,2022,Port Royal Golf Course,528,Butterfield Bermuda Championship,2021-10-31
378,2023,Port Royal Golf Course,528,Butterfield Bermuda Championship,2022-10-30
642,2024,Port Royal Golf Course,R2024528,Butterfield Bermuda Championship,2024-11-10


### Cut Percentage and FedEx Points
Use a rolling-window approach to look at the most recent cut percentage and how many FedEx cup points have been accumulated recently. This will intentionally not match the PGA Tour stats that start over every year, but will have the same amount of data all the time.  We also add a new feature called Form Density which divides the FedEx Cup Points by the Total Events.

In [3]:
importlib.reload(utils.db_utils)
from utils.db_utils import get_cut_and_fedex_history

cuts = get_cut_and_fedex_history("data/golf.db", history_df, window_months=9)
# cuts["2024-05-12"].head(20)

for end_date, df in cuts.items():
    print(f"\n📆 {end_date} — {df['TOURNAMENT'].iloc[0]} ({len(df)} players)")
    display(df.head(3))


📆 2019-11-03 — Bermuda Championship (580 players)


,PLAYER,TOTAL_EVENTS_PLAYED,CUTS_MADE,FEDEX_CUP_POINTS,CUT_PERCENTAGE,form_density,CONSECUTIVE_CUTS,ENDING_DATE,TOURNAMENT
0,Aaron Baddeley,14,7,161.95,50.0,11.57,0,2019-11-03,Bermuda Championship
1,Aaron Rai,1,1,0.00,100.0,0.00,1,2019-11-03,Bermuda Championship
2,Aaron Wise,15,10,211.82,66.7,14.12,0,2019-11-03,Bermuda Championship



📆 2020-11-01 — Bermuda Championship (403 players)


,PLAYER,TOTAL_EVENTS_PLAYED,CUTS_MADE,FEDEX_CUP_POINTS,CUT_PERCENTAGE,form_density,CONSECUTIVE_CUTS,ENDING_DATE,TOURNAMENT
0,Aaron Baddeley,10,3,82.17,30.0,8.22,0,2020-11-01,Bermuda Championship
1,Aaron Crawford,1,0,0.00,0.0,0.00,0,2020-11-01,Bermuda Championship
2,Aaron Wise,12,4,79.25,33.3,6.60,0,2020-11-01,Bermuda Championship



📆 2021-10-31 — Butterfield Bermuda Championship (554 players)


,PLAYER,TOTAL_EVENTS_PLAYED,CUTS_MADE,FEDEX_CUP_POINTS,CUT_PERCENTAGE,form_density,CONSECUTIVE_CUTS,ENDING_DATE,TOURNAMENT
0,Aaron Baddeley,10,3,35.09,30.0,3.51,1,2021-10-31,Butterfield Bermuda Championship
1,Aaron Pike,1,0,0.00,0.0,0.00,0,2021-10-31,Butterfield Bermuda Championship
2,Aaron Rai,6,2,0.00,33.3,0.00,0,2021-10-31,Butterfield Bermuda Championship



📆 2022-10-30 — Butterfield Bermuda Championship (594 players)


,PLAYER,TOTAL_EVENTS_PLAYED,CUTS_MADE,FEDEX_CUP_POINTS,CUT_PERCENTAGE,form_density,CONSECUTIVE_CUTS,ENDING_DATE,TOURNAMENT
0,A.J. Ewart,1,0,0.00,0.0,0.00,0,2022-10-30,Butterfield Bermuda Championship
1,Aaron Baddeley,8,4,70.94,50.0,8.87,1,2022-10-30,Butterfield Bermuda Championship
2,Aaron Beverly,1,0,0.00,0.0,0.00,0,2022-10-30,Butterfield Bermuda Championship



📆 2024-11-10 — Butterfield Bermuda Championship (561 players)


,PLAYER,TOTAL_EVENTS_PLAYED,CUTS_MADE,FEDEX_CUP_POINTS,CUT_PERCENTAGE,form_density,CONSECUTIVE_CUTS,ENDING_DATE,TOURNAMENT
0,Aaron Baddeley,16,9,142.188,56.2,8.89,0,2024-11-10,Butterfield Bermuda Championship
1,Aaron Cockerill,2,0,0.000,0.0,0.00,0,2024-11-10,Butterfield Bermuda Championship
2,Aaron Rai,18,16,1741.132,88.9,96.73,14,2024-11-10,Butterfield Bermuda Championship


### Recent Form

In [4]:
importlib.reload(utils.db_utils)
from utils.db_utils import get_recent_avg_finish

recent_form = get_recent_avg_finish("data/golf.db", history_df, window_months=9)

# Example preview
for date, df in recent_form.items(): 
    print(f"\n📆 {date} — {df['TOURNAMENT'].iloc[0]} ({len(df)} players)")
    display(df.head(3))


📆 2019-11-03 — Bermuda Championship (580 players)


,PLAYER,TOTAL_EVENTS_PLAYED,RECENT_FORM,adj_form,ENDING_DATE,TOURNAMENT
0,Robert MacIntyre,1,6.0,8.66,2019-11-03,Bermuda Championship
1,Lanto Griffin,4,11.8,7.33,2019-11-03,Bermuda Championship
2,Aaron Rai,1,12.0,17.31,2019-11-03,Bermuda Championship



📆 2020-11-01 — Bermuda Championship (403 players)


,PLAYER,TOTAL_EVENTS_PLAYED,RECENT_FORM,adj_form,ENDING_DATE,TOURNAMENT
0,Xander Schauffele,13,17.5,6.63,2020-11-01,Bermuda Championship
1,Daniel Berger,12,19.3,7.52,2020-11-01,Bermuda Championship
2,Bryson DeChambeau,13,20.2,7.65,2020-11-01,Bermuda Championship



📆 2021-10-31 — Butterfield Bermuda Championship (554 players)


,PLAYER,TOTAL_EVENTS_PLAYED,RECENT_FORM,adj_form,ENDING_DATE,TOURNAMENT
0,Harry Hall,1,8.0,11.54,2021-10-31,Butterfield Bermuda Championship
1,Marcel Siem,1,15.0,21.64,2021-10-31,Butterfield Bermuda Championship
2,Grant Hirschman,1,17.0,24.53,2021-10-31,Butterfield Bermuda Championship



📆 2022-10-30 — Butterfield Bermuda Championship (594 players)


,PLAYER,TOTAL_EVENTS_PLAYED,RECENT_FORM,adj_form,ENDING_DATE,TOURNAMENT
0,Ryo Hisatsune,1,12.0,17.31,2022-10-30,Butterfield Bermuda Championship
1,Anthony Quayle,1,15.0,21.64,2022-10-30,Butterfield Bermuda Championship
2,Mikumu Horikawa,1,16.0,23.08,2022-10-30,Butterfield Bermuda Championship



📆 2024-11-10 — Butterfield Bermuda Championship (561 players)


,PLAYER,TOTAL_EVENTS_PLAYED,RECENT_FORM,adj_form,ENDING_DATE,TOURNAMENT
0,Scottie Scheffler,15,5.8,2.09,2024-11-10,Butterfield Bermuda Championship
1,Yuta Sugiura,1,6.0,8.66,2024-11-10,Butterfield Bermuda Championship
2,Taiga Semikawa,1,9.0,12.98,2024-11-10,Butterfield Bermuda Championship


### Course History

In [5]:
importlib.reload(utils.db_utils)
from utils.db_utils import get_course_history

# Filter history_df for only the course we're targeting
target_course = tournament_config["new"]["course"]
course_df = history_df[history_df["COURSE"] == target_course]
course_hist = get_course_history("data/golf.db", course_df)

# View example
for date, df in course_hist.items():
    if not df.empty:
        print(f"\n🏌️‍♂️ Course history for {df['TOURNAMENT'].iloc[0]} on {date}")
        display(df.head(3))


🏌️‍♂️ Course history for Bermuda Championship on 2020-11-01


,PLAYER,TOTAL_EVENTS_PLAYED,COURSE_HISTORY,adj_ch,ENDING_DATE,COURSE,TOURNAMENT
0,Aaron Wise,1,3.0,4.33,2020-11-01,Port Royal Golf Course,Bermuda Championship
1,Alex Cejka,1,58.0,83.68,2020-11-01,Port Royal Golf Course,Bermuda Championship
2,Alex Noren,1,15.0,21.64,2020-11-01,Port Royal Golf Course,Bermuda Championship



🏌️‍♂️ Course history for Butterfield Bermuda Championship on 2021-10-31


,PLAYER,TOTAL_EVENTS_PLAYED,COURSE_HISTORY,adj_ch,ENDING_DATE,COURSE,TOURNAMENT
0,Aaron Baddeley,1,90.0,129.84,2021-10-31,Port Royal Golf Course,Butterfield Bermuda Championship
1,Aaron Wise,2,14.5,13.20,2021-10-31,Port Royal Golf Course,Butterfield Bermuda Championship
2,Adam Schenk,1,26.0,37.51,2021-10-31,Port Royal Golf Course,Butterfield Bermuda Championship



🏌️‍♂️ Course history for Butterfield Bermuda Championship on 2022-10-30


,PLAYER,TOTAL_EVENTS_PLAYED,COURSE_HISTORY,adj_ch,ENDING_DATE,COURSE,TOURNAMENT
0,Aaron Baddeley,1,90.0,129.84,2022-10-30,Port Royal Golf Course,Butterfield Bermuda Championship
1,Aaron Rai,1,62.0,89.45,2022-10-30,Port Royal Golf Course,Butterfield Bermuda Championship
2,Aaron Wise,2,14.5,13.20,2022-10-30,Port Royal Golf Course,Butterfield Bermuda Championship



🏌️‍♂️ Course history for Butterfield Bermuda Championship on 2024-11-10


,PLAYER,TOTAL_EVENTS_PLAYED,COURSE_HISTORY,adj_ch,ENDING_DATE,COURSE,TOURNAMENT
0,Aaron Baddeley,2,48.0,43.69,2024-11-10,Port Royal Golf Course,Butterfield Bermuda Championship
1,Aaron Jarvis,1,90.0,129.84,2024-11-10,Port Royal Golf Course,Butterfield Bermuda Championship
2,Aaron Rai,2,58.0,52.79,2024-11-10,Port Royal Golf Course,Butterfield Bermuda Championship


In [6]:
#  DEBUGGING: Check for empty DataFrames or missing columns
for date_key, df in course_hist.items():
    if df.empty:
        print(f"ℹ️ Empty course_hist for date {date_key}")
    elif "PLAYER" not in df.columns:
        print(f"❌ Missing 'PLAYER' column in course_hist[{date_key}]. Columns present: {df.columns.tolist()}")
        display(df.head())

ℹ️ Empty course_hist for date 2019-11-03


## Training Dataset

In [7]:
pd.set_option("display.max_columns", None)   # Show all columns
importlib.reload(utils.db_utils)
from utils.db_utils import build_training_rows
training_df = build_training_rows(
    db_path,
    history_df,
    cuts,
    recent_form,
    course_hist,
)
training_df.head(10)
# training_df.info()
# training_df.isna().sum().sort_values(ascending=False)

,SEASON,ENDING_DATE,TOURNAMENT,COURSE,PLAYER,POS,FINAL_POS,SGTTG_RANK,SGTTG,SGOTT_RANK,SGOTT,SGAPR_RANK,SGAPR,SGATG_RANK,SGATG,SGP_RANK,SGP,BIRDIES_RANK,BIRDIES,PAR_3_RANK,PAR_3,PAR_4_RANK,PAR_4,PAR_5_RANK,PAR_5,TOTAL_DRIVING_RANK,TOTAL_DRIVING,DRIVING_DISTANCE_RANK,DRIVING_DISTANCE,DRIVING_ACCURACY_RANK,DRIVING_ACCURACY,GIR_RANK,GIR,SCRAMBLING_RANK,SCRAMBLING,OWGR_RANK,OWGR,VEGAS_ODDS,CUT_PERCENTAGE,FEDEX_CUP_POINTS,form_density,CONSECUTIVE_CUTS,RECENT_FORM,adj_form,COURSE_HISTORY,adj_ch,TOP_20
0,2020,2019-11-03,Bermuda Championship,Port Royal Golf Course,Brendon Todd,1,1,127.0,-0.101,124.0,-0.073,123.0,-0.119,70.0,0.092,20.0,0.498,68.0,22.03,12.0,2.98,21.0,3.98,123.0,4.66,91.0,191.0,187.0,282.7,4.0,71.36,138.0,65.5,3.0,66.1,39.0,2.74,80.0,36.4,111.72,10.16,1.0,67.9,27.32,NaN,NaN,1
1,2020,2019-11-03,Bermuda Championship,Port Royal Golf Course,Harry Higgs,2,2,139.0,-0.202,35.0,0.307,161.0,-0.361,144.0,-0.148,89.0,0.080,134.0,20.82,171.0,3.12,21.0,3.98,165.0,4.72,26.0,135.0,63.0,301.0,72.0,62.19,122.0,66.06,62.0,60.55,132.0,1.26,50.0,25.0,31.80,7.95,0.0,73.3,45.54,NaN,NaN,1
2,2020,2019-11-03,Bermuda Championship,Port Royal Golf Course,Brian Gay,T3,3,177.0,-0.707,173.0,-0.353,148.0,-0.293,117.0,-0.061,72.0,0.129,65.0,22.05,143.0,3.08,75.0,4.01,165.0,4.72,188.0,285.0,162.0,288.5,123.0,58.68,158.0,64.81,143.0,56.89,276.0,0.59,40.0,69.6,359.70,15.64,4.0,58.4,18.38,NaN,NaN,1
3,2020,2019-11-03,Bermuda Championship,Port Royal Golf Course,Hank Lebioda,T3,3,117.0,-0.006,121.0,-0.062,115.0,-0.053,62.0,0.109,182.0,-0.615,135.0,20.81,55.0,3.03,166.0,4.06,123.0,4.66,131.0,217.0,121.0,294.7,96.0,60.34,79.0,67.68,158.0,55.94,309.0,0.52,80.0,68.8,176.04,11.00,0.0,60.1,21.21,NaN,NaN,1
4,2020,2019-11-03,Bermuda Championship,Port Royal Golf Course,Scottie Scheffler,T3,3,10.0,1.131,10.0,0.618,33.0,0.398,58.0,0.115,117.0,-0.053,7.0,24.93,71.0,3.04,6.0,3.95,21.0,4.55,7.0,100.0,16.0,308.9,84.0,61.24,43.0,69.04,58.0,60.61,29.0,3.55,20.0,83.3,76.80,12.80,3.0,43.8,22.51,NaN,NaN,1
5,2020,2019-11-03,Bermuda Championship,Port Royal Golf Course,Aaron Wise,T3,3,95.0,0.110,100.0,0.016,89.0,0.104,98.0,-0.010,171.0,-0.479,48.0,22.35,16.0,2.99,155.0,4.05,37.0,4.58,56.0,162.0,59.0,301.5,103.0,59.94,3.0,72.1,157.0,56.19,212.0,0.80,20.0,66.7,211.82,14.12,0.0,53.8,19.40,NaN,NaN,1
6,2020,2019-11-03,Bermuda Championship,Port Royal Golf Course,Fabian Gomez,7,7,141.0,-0.259,162.0,-0.251,130.0,-0.137,52.0,0.129,103.0,0.011,135.0,20.81,106.0,3.06,21.0,3.98,162.0,4.71,88.0,188.0,160.0,289.2,28.0,65.8,108.0,66.67,49.0,60.91,246.0,0.68,50.0,64.3,221.26,15.80,5.0,56.9,21.01,NaN,NaN,1
7,2020,2019-11-03,Bermuda Championship,Port Royal Golf Course,Ryan Armour,T8,8,76.0,0.225,99.0,0.020,100.0,0.044,46.0,0.160,166.0,-0.418,54.0,22.22,9.0,2.97,121.0,4.03,112.0,4.65,80.0,184.0,182.0,284.1,2.0,73.86,74.0,67.91,151.0,56.42,184.0,0.91,50.0,47.8,297.90,12.95,1.0,64.3,20.23,NaN,NaN,1
8,2020,2019-11-03,Bermuda Championship,Port Royal Golf Course,David Hearn,T8,8,151.0,-0.402,158.0,-0.220,142.0,-0.229,84.0,0.047,77.0,0.120,174.0,19.61,128.0,3.07,54.0,4.00,149.0,4.69,110.0,200.0,188.0,281.8,12.0,68.44,102.0,66.88,15.0,64.14,542.0,0.27,60.0,41.7,73.49,6.12,0.0,70.1,27.33,NaN,NaN,1
9,2020,2019-11-03,Bermuda Championship,Port Royal Golf Course,Wes Roach,T8,8,167.0,-0.551,151.0,-0.175,178.0,-0.449,78.0,0.073,175.0,-0.505,81.0,21.81,106.0,3.06,155.0,4.05,43.0,4.59,127.0,214.0,149.0,290.5,65.0,62.61,73.0,67.92,112.0,58.01,310.0,0.52,100.0,66.7,330.56,27.55,1.0,54.3,21.17,NaN,NaN,1


In [8]:
# Play with different years to see if I notice anything wrong (check Odds for tournament name mismatches)
training_df[training_df["SEASON"] == 2024].head()

,SEASON,ENDING_DATE,TOURNAMENT,COURSE,PLAYER,POS,FINAL_POS,SGTTG_RANK,SGTTG,SGOTT_RANK,SGOTT,SGAPR_RANK,SGAPR,SGATG_RANK,SGATG,SGP_RANK,SGP,BIRDIES_RANK,BIRDIES,PAR_3_RANK,PAR_3,PAR_4_RANK,PAR_4,PAR_5_RANK,PAR_5,TOTAL_DRIVING_RANK,TOTAL_DRIVING,DRIVING_DISTANCE_RANK,DRIVING_DISTANCE,DRIVING_ACCURACY_RANK,DRIVING_ACCURACY,GIR_RANK,GIR,SCRAMBLING_RANK,SCRAMBLING,OWGR_RANK,OWGR,VEGAS_ODDS,CUT_PERCENTAGE,FEDEX_CUP_POINTS,form_density,CONSECUTIVE_CUTS,RECENT_FORM,adj_form,COURSE_HISTORY,adj_ch,TOP_20
510,2024,2024-11-10,Butterfield Bermuda Championship,Port Royal Golf Course,Rafael Campos,1,1,153.0,-0.486,137.0,-0.194,96.0,0.067,167.0,-0.359,144.0,-0.259,104.0,22.01%,162.0,3.10,131.0,4.04,90.0,4.58,97.0,178.0,54.0,306.0,124.0,58.06%,51.0,68.59%,138.0,56.80%,304.0,0.4629,NaN,20.0,134.115,8.94,0.0,76.0,27.41,54.0,49.15,1
511,2024,2024-11-10,Butterfield Bermuda Championship,Port Royal Golf Course,Andrew Novak,2,2,21.0,0.766,63.0,0.172,20.0,0.455,59.0,0.139,86.0,0.040,55.0,23.12%,6.0,2.98,26.0,3.98,90.0,4.58,146.0,214.0,102.0,300.1,112.0,58.79%,17.0,70.58%,41.0,62.23%,155.0,0.8783,NaN,78.9,589.493,31.03,1.0,40.5,13.52,50.7,36.57,1
512,2024,2024-11-10,Butterfield Bermuda Championship,Port Royal Golf Course,Adrien Dumont de Chassart,T3,3,169.0,-1.094,157.0,-0.381,157.0,-0.421,157.0,-0.292,141.0,-0.244,141.0,21.06%,142.0,3.08,141.0,4.05,116.0,4.61,154.0,229.0,69.0,303.9,160.0,54.73%,104.0,66.23%,171.0,53.47%,263.0,0.5384,NaN,37.5,49.505,3.09,0.0,76.4,26.97,NaN,NaN,1
513,2024,2024-11-10,Butterfield Bermuda Championship,Port Royal Golf Course,Mark Hubbard,T3,3,66.0,0.299,95.0,0.044,48.0,0.275,108.0,-0.021,90.0,0.023,52.0,23.20%,126.0,3.07,43.0,3.99,27.0,4.51,56.0,156.0,138.0,295.7,18.0,66.89%,80.0,67.31%,92.0,59.57%,105.0,1.2098,NaN,81.8,332.979,15.14,2.0,53.2,16.97,43.3,26.90,1
514,2024,2024-11-10,Butterfield Bermuda Championship,Port Royal Golf Course,Vince Whaley,T5,5,162.0,-0.791,170.0,-0.656,155.0,-0.405,22.0,0.270,14.0,0.496,48.0,23.26%,114.0,3.06,10.0,3.97,129.0,4.63,148.0,219.0,45.0,307.7,174.0,51.80%,138.0,64.91%,63.0,61.21%,120.0,1.1119,NaN,64.3,150.740,10.77,3.0,61.9,22.86,35.0,25.25,1


### Normalization
***Fix Historical Odds***

Many names do not have historical odds but rather are part of the "field." So giving these "NaN" values the average odds (as I would for missing stats) isn't a good approximation of reality – they are typically the "field" because they are not notable and have poor odds individually.

There are also some crazy odds numbers occasionally (1000/1 or 3000/1) that don't happen all that much and are making that tail too long.

This cell assigns odds of 1000/1 for anyone missing and clips anything larger to 1000/1 to normalize the data better and assign the missing odds more appropriately where they belong.

***OWGR Adjustment***

The NaN's for OWGR are similarly bad players.  So we should assign these the worst ranking instead of the mean and clip it at 1000 just in case there are ever outliers.

For the OWGR score, it should match the lowest score in the dataset to assign that person the same as the worst.

***Recent Form Adjustment***

The NaN's for Recent Form mean that this player has not played any tournaments in the lookback period (9 months).  Similar to Odds and OWGR, I want to punish those that don't play often in my model.  Set these to 90 (i.e. like they miss a lot of cuts - because if they aren't playing they aren't good enough to make it to these tournaments very frequently).  The adj_form feature will also need to be updated here with the new data.

***FedEx Cup Point Adjustment***

The NaN's for FedEx Cup Points means there is no data.  Therefore they should be 0.

In [9]:
# === VEGAS ODDS ===
training_df["VEGAS_ODDS"] = training_df["VEGAS_ODDS"].fillna(1000).clip(upper=1000)

# === OWGR and OWGR_RANK ===
owgr_min = training_df["OWGR"].min(skipna=True)
training_df["OWGR"] = training_df["OWGR"].fillna(owgr_min)
training_df["OWGR_RANK"] = training_df["OWGR_RANK"].fillna(1000).astype(float).clip(upper=1000)

# === RECENT FORM and adj_form ===
training_df["RECENT_FORM"] = training_df["RECENT_FORM"].fillna(90)
if "TOTAL_EVENTS_PLAYED" in training_df.columns:
    training_df["adj_form"] = (
        training_df["RECENT_FORM"] / np.log1p(training_df["TOTAL_EVENTS_PLAYED"])
    ).round(2)

# === FEDEX CUP POINTS ===
training_df["FEDEX_CUP_POINTS"] = training_df["FEDEX_CUP_POINTS"].fillna(0)

training_df.head(5)


,SEASON,ENDING_DATE,TOURNAMENT,COURSE,PLAYER,POS,FINAL_POS,SGTTG_RANK,SGTTG,SGOTT_RANK,SGOTT,SGAPR_RANK,SGAPR,SGATG_RANK,SGATG,SGP_RANK,SGP,BIRDIES_RANK,BIRDIES,PAR_3_RANK,PAR_3,PAR_4_RANK,PAR_4,PAR_5_RANK,PAR_5,TOTAL_DRIVING_RANK,TOTAL_DRIVING,DRIVING_DISTANCE_RANK,DRIVING_DISTANCE,DRIVING_ACCURACY_RANK,DRIVING_ACCURACY,GIR_RANK,GIR,SCRAMBLING_RANK,SCRAMBLING,OWGR_RANK,OWGR,VEGAS_ODDS,CUT_PERCENTAGE,FEDEX_CUP_POINTS,form_density,CONSECUTIVE_CUTS,RECENT_FORM,adj_form,COURSE_HISTORY,adj_ch,TOP_20
0,2020,2019-11-03,Bermuda Championship,Port Royal Golf Course,Brendon Todd,1,1,127.0,-0.101,124.0,-0.073,123.0,-0.119,70.0,0.092,20.0,0.498,68.0,22.03,12.0,2.98,21.0,3.98,123.0,4.66,91.0,191.0,187.0,282.7,4.0,71.36,138.0,65.5,3.0,66.1,39.0,2.74,80.0,36.4,111.72,10.16,1.0,67.9,27.32,NaN,NaN,1
1,2020,2019-11-03,Bermuda Championship,Port Royal Golf Course,Harry Higgs,2,2,139.0,-0.202,35.0,0.307,161.0,-0.361,144.0,-0.148,89.0,0.080,134.0,20.82,171.0,3.12,21.0,3.98,165.0,4.72,26.0,135.0,63.0,301.0,72.0,62.19,122.0,66.06,62.0,60.55,132.0,1.26,50.0,25.0,31.80,7.95,0.0,73.3,45.54,NaN,NaN,1
2,2020,2019-11-03,Bermuda Championship,Port Royal Golf Course,Brian Gay,T3,3,177.0,-0.707,173.0,-0.353,148.0,-0.293,117.0,-0.061,72.0,0.129,65.0,22.05,143.0,3.08,75.0,4.01,165.0,4.72,188.0,285.0,162.0,288.5,123.0,58.68,158.0,64.81,143.0,56.89,276.0,0.59,40.0,69.6,359.70,15.64,4.0,58.4,18.38,NaN,NaN,1
3,2020,2019-11-03,Bermuda Championship,Port Royal Golf Course,Hank Lebioda,T3,3,117.0,-0.006,121.0,-0.062,115.0,-0.053,62.0,0.109,182.0,-0.615,135.0,20.81,55.0,3.03,166.0,4.06,123.0,4.66,131.0,217.0,121.0,294.7,96.0,60.34,79.0,67.68,158.0,55.94,309.0,0.52,80.0,68.8,176.04,11.00,0.0,60.1,21.21,NaN,NaN,1
4,2020,2019-11-03,Bermuda Championship,Port Royal Golf Course,Scottie Scheffler,T3,3,10.0,1.131,10.0,0.618,33.0,0.398,58.0,0.115,117.0,-0.053,7.0,24.93,71.0,3.04,6.0,3.95,21.0,4.55,7.0,100.0,16.0,308.9,84.0,61.24,43.0,69.04,58.0,60.61,29.0,3.55,20.0,83.3,76.80,12.80,3.0,43.8,22.51,NaN,NaN,1


#### Average the NaNs
At this point, the intentional offsetting of NaNs for poor players is over.  I don't want to penalize statistics or course history because you never know where they might land on the spectrum, so for these NaN values, we will take the mean.

In [10]:
# === Clean percentage stats stored as strings like '62.5%' ===
percent_stats = ["SCRAMBLING", "DRIVING_ACCURACY", "BIRDIES", "GIR"]

for col in percent_stats:
    if col in training_df.columns:
        training_df[col] = (
            training_df[col]
            .astype(str)
            .str.replace('%', '', regex=False)
            .replace(['None', 'nan', 'NaN', '--', 'DNP', ''], np.nan)
            .astype(float)
        )

# Select only the numeric columns
numeric_columns = training_df.select_dtypes(include=['float64', 'int64'])

# Fill NaN values with the mean of each column
numeric_columns = numeric_columns.fillna(numeric_columns.mean())

# Update the original dataframe with the filled numeric columns
training_df.update(numeric_columns)

training_df.head()

,SEASON,ENDING_DATE,TOURNAMENT,COURSE,PLAYER,POS,FINAL_POS,SGTTG_RANK,SGTTG,SGOTT_RANK,SGOTT,SGAPR_RANK,SGAPR,SGATG_RANK,SGATG,SGP_RANK,SGP,BIRDIES_RANK,BIRDIES,PAR_3_RANK,PAR_3,PAR_4_RANK,PAR_4,PAR_5_RANK,PAR_5,TOTAL_DRIVING_RANK,TOTAL_DRIVING,DRIVING_DISTANCE_RANK,DRIVING_DISTANCE,DRIVING_ACCURACY_RANK,DRIVING_ACCURACY,GIR_RANK,GIR,SCRAMBLING_RANK,SCRAMBLING,OWGR_RANK,OWGR,VEGAS_ODDS,CUT_PERCENTAGE,FEDEX_CUP_POINTS,form_density,CONSECUTIVE_CUTS,RECENT_FORM,adj_form,COURSE_HISTORY,adj_ch,TOP_20
0,2020,2019-11-03,Bermuda Championship,Port Royal Golf Course,Brendon Todd,1,1,127.0,-0.101,124.0,-0.073,123.0,-0.119,70.0,0.092,20.0,0.498,68.0,22.03,12.0,2.98,21.0,3.98,123.0,4.66,91.0,191.0,187.0,282.7,4.0,71.36,138.0,65.50,3.0,66.10,39.0,2.74,80.0,36.4,111.72,10.16,1.0,67.9,27.32,53.568792,62.850638,1
1,2020,2019-11-03,Bermuda Championship,Port Royal Golf Course,Harry Higgs,2,2,139.0,-0.202,35.0,0.307,161.0,-0.361,144.0,-0.148,89.0,0.080,134.0,20.82,171.0,3.12,21.0,3.98,165.0,4.72,26.0,135.0,63.0,301.0,72.0,62.19,122.0,66.06,62.0,60.55,132.0,1.26,50.0,25.0,31.80,7.95,0.0,73.3,45.54,53.568792,62.850638,1
2,2020,2019-11-03,Bermuda Championship,Port Royal Golf Course,Brian Gay,T3,3,177.0,-0.707,173.0,-0.353,148.0,-0.293,117.0,-0.061,72.0,0.129,65.0,22.05,143.0,3.08,75.0,4.01,165.0,4.72,188.0,285.0,162.0,288.5,123.0,58.68,158.0,64.81,143.0,56.89,276.0,0.59,40.0,69.6,359.70,15.64,4.0,58.4,18.38,53.568792,62.850638,1
3,2020,2019-11-03,Bermuda Championship,Port Royal Golf Course,Hank Lebioda,T3,3,117.0,-0.006,121.0,-0.062,115.0,-0.053,62.0,0.109,182.0,-0.615,135.0,20.81,55.0,3.03,166.0,4.06,123.0,4.66,131.0,217.0,121.0,294.7,96.0,60.34,79.0,67.68,158.0,55.94,309.0,0.52,80.0,68.8,176.04,11.00,0.0,60.1,21.21,53.568792,62.850638,1
4,2020,2019-11-03,Bermuda Championship,Port Royal Golf Course,Scottie Scheffler,T3,3,10.0,1.131,10.0,0.618,33.0,0.398,58.0,0.115,117.0,-0.053,7.0,24.93,71.0,3.04,6.0,3.95,21.0,4.55,7.0,100.0,16.0,308.9,84.0,61.24,43.0,69.04,58.0,60.61,29.0,3.55,20.0,83.3,76.80,12.80,3.0,43.8,22.51,53.568792,62.850638,1


In [11]:
# Check for NaN values
assert training_df.isna().sum().sum() == 0, "🚨 Still missing values!"

In [12]:
# Inspect which columns have NaN values
training_df.isna().sum().sort_values(ascending=False).head(10)


SEASON         0
ENDING_DATE    0
TOURNAMENT     0
COURSE         0
PLAYER         0
POS            0
FINAL_POS      0
SGTTG_RANK     0
SGTTG          0
SGOTT_RANK     0
dtype: int64

### Check Features for Outliers and General Health (Histograms)
Now that all the data is cleaned, let's look at the distribution graphs.

In [19]:
import plotly.express as px
import plotly.subplots as sp
import plotly.graph_objects as go
import pandas as pd
import numpy as np

# Select numeric columns with >1 unique value, excluding label
df_numeric = training_df.select_dtypes(include=np.number)
df_numeric = df_numeric.loc[:, df_numeric.nunique() > 1]
df_numeric = df_numeric.drop(columns=["TOP_20"], errors="ignore")

# Create subplots
n_cols = 3
n_rows = int(np.ceil(len(df_numeric.columns) / n_cols))
fig = sp.make_subplots(rows=n_rows, cols=n_cols, subplot_titles=df_numeric.columns)

# Add one histogram per feature with smart binning
for i, column in enumerate(df_numeric.columns):
    row = i // n_cols + 1
    col = i % n_cols + 1

    # Compute dynamic bin count
    col_range = df_numeric[column].max() - df_numeric[column].min()
    n_unique = df_numeric[column].nunique()
    nbins = min(50, max(5, int(n_unique / 2))) if col_range < 10 else 50  # tighter for short-range

    fig.add_trace(
        go.Histogram(
            x=df_numeric[column],
            nbinsx=nbins,
            histnorm="density",
            marker=dict(color="#636EFA"),
            opacity=0.8
        ),
        row=row,
        col=col
    )

# Style the figure
fig.update_layout(
    height=300 * n_rows,
    width=1000,
    title_text="Feature Distributions (with Adaptive Binning)",
    template="plotly_dark",
    showlegend=False
)

fig.show()

# Current Week Data

## Import 2025 Field
**Important!**  Save curren't week's `DKSalaries.csv` into Data folder before this step.

In [20]:
importlib.reload(utils.db_utils)
from utils.db_utils import DK_PLAYER_NAME_MAP
from utils.db_utils import standardize_player_names

# Load DraftKings player list
dk = pd.read_csv("data/DKSalaries.csv", usecols=["Name", "Salary"])

# Normalize DK names to PGA naming convention
dk["Name"] = dk["Name"].replace(DK_PLAYER_NAME_MAP)

# Rename for consistency
dk = dk.rename(columns={"Name": "PLAYER", "Salary": "SALARY"})

dk = standardize_player_names(dk)  # ✅ Normalize to match DB

dk.head(10)

,PLAYER,SALARY
0,Rico Hoey,10400
1,Kevin Yu,10300
2,Thorbjorn Olesen,10100
3,Nicolas Echavarria,9700
4,Pierceson Coody,9600
5,Patrick Rodgers,9500
6,Matti Schmid,9400
7,Michael Brennan,9300
8,Jesper Svensson,9200
9,Matt Wallace,9100


## Vegas Odds
Confirm the [Vegas Odds](http://golfodds.com/weekly-odds.html) page is updated with current tournament.

Note: sometimes this site can give errors and not load correctly.  Just have to do it again later.
Another trick to try is change the url try to run it and then put the url back the way it was.  Worked once, maybe random coincidence.

In [14]:
importlib.reload(utils.db_utils)
from utils.db_utils import get_current_week_odds

odds_current = get_current_week_odds(season=2025, tournament_name=new_tournament_name)
odds_current.head(10)


,SEASON,TOURNAMENT,PLAYER,ODDS,VEGAS_ODDS
1,2025,Butterfield Bermuda Championship,Rico Hoey,20/1,20.0
2,2025,Butterfield Bermuda Championship,Kevin Yu,25/1,25.0
3,2025,Butterfield Bermuda Championship,Thorbjorn Olesen,25/1,25.0
4,2025,Butterfield Bermuda Championship,Nico Echavarria,25/1,25.0
5,2025,Butterfield Bermuda Championship,Pierceson Coody,30/1,30.0
6,2025,Butterfield Bermuda Championship,Patrick Rodgers,30/1,30.0
7,2025,Butterfield Bermuda Championship,Matt Kuchar,30/1,30.0
8,2025,Butterfield Bermuda Championship,Michael Brennan,30/1,30.0
9,2025,Butterfield Bermuda Championship,Vince Whaley,30/1,30.0
10,2025,Butterfield Bermuda Championship,Matt Wallace,30/1,30.0


## Cut Percentage and FedEx Points

In [15]:
importlib.reload(utils.db_utils)
from utils.db_utils import get_cut_and_fedex_history

cuts_rolling = get_cut_and_fedex_history(db_path, this_week_history, window_months=9)
cuts_current = cuts_rolling[this_week_key].copy()
cuts_current.head(10)

,PLAYER,TOTAL_EVENTS_PLAYED,CUTS_MADE,FEDEX_CUP_POINTS,CUT_PERCENTAGE,form_density,CONSECUTIVE_CUTS,ENDING_DATE,TOURNAMENT
0,A.J. Ewart,1,0,0.000,0.0,0.00,0,2025-11-16,Butterfield Bermuda Championship
1,Aaron Baddeley,10,3,21.950,30.0,2.19,1,2025-11-16,Butterfield Bermuda Championship
2,Aaron Cockerill,1,0,0.000,0.0,0.00,0,2025-11-16,Butterfield Bermuda Championship
3,Aaron Rai,17,14,861.355,82.4,50.67,6,2025-11-16,Butterfield Bermuda Championship
4,Aaron Wise,6,1,6.333,16.7,1.06,0,2025-11-16,Butterfield Bermuda Championship
5,Adam Hadwin,20,10,110.038,50.0,5.50,1,2025-11-16,Butterfield Bermuda Championship
6,Adam Schenk,19,7,173.425,36.8,9.13,4,2025-11-16,Butterfield Bermuda Championship
7,Adam Scott,16,13,378.938,81.2,23.68,3,2025-11-16,Butterfield Bermuda Championship
8,Adam Svensson,18,8,131.550,44.4,7.31,2,2025-11-16,Butterfield Bermuda Championship
9,Adrian Otaegui,1,0,0.000,0.0,0.00,0,2025-11-16,Butterfield Bermuda Championship


## Recent Form

In [16]:
importlib.reload(utils.db_utils)
from utils.db_utils import get_recent_avg_finish

recent_form = get_recent_avg_finish(db_path, this_week_history, window_months=9)
recent_form_current = recent_form[this_week_key].copy()

recent_form_current.head(10)

,PLAYER,TOTAL_EVENTS_PLAYED,RECENT_FORM,adj_form,ENDING_DATE,TOURNAMENT
0,Scottie Scheffler,17,4.8,1.66,2025-11-16,Butterfield Bermuda Championship
1,Jason Scrivener,1,8.0,11.54,2025-11-16,Butterfield Bermuda Championship
2,Bubba Watson,1,14.0,20.20,2025-11-16,Butterfield Bermuda Championship
3,Doc Redman,2,14.5,13.20,2025-11-16,Butterfield Bermuda Championship
4,Jon Rahm,4,15.8,9.82,2025-11-16,Butterfield Bermuda Championship
5,Robin Williams,1,17.0,24.53,2025-11-16,Butterfield Bermuda Championship
6,Andy Sullivan,1,17.0,24.53,2025-11-16,Butterfield Bermuda Championship
7,Rory McIlroy,13,17.6,6.67,2025-11-16,Butterfield Bermuda Championship
8,Kazuki Higa,1,18.0,25.97,2025-11-16,Butterfield Bermuda Championship
9,Joel Girrbach,1,20.0,28.85,2025-11-16,Butterfield Bermuda Championship


## Course History

In [17]:
importlib.reload(utils.db_utils)
from utils.db_utils import get_course_history

course_hist = get_course_history(db_path, this_week_history, lookback_years=7)
course_hist_current = course_hist[this_week_key].copy()

course_hist_current.head(10)


,PLAYER,TOTAL_EVENTS_PLAYED,COURSE_HISTORY,adj_ch,ENDING_DATE,COURSE,TOURNAMENT
0,Aaron Baddeley,3,55.3,39.89,2025-11-16,Port Royal Golf Course,Butterfield Bermuda Championship
1,Aaron Jarvis,1,90.0,129.84,2025-11-16,Port Royal Golf Course,Butterfield Bermuda Championship
2,Aaron Rai,2,58.0,52.79,2025-11-16,Port Royal Golf Course,Butterfield Bermuda Championship
3,Aaron Wise,2,14.5,13.20,2025-11-16,Port Royal Golf Course,Butterfield Bermuda Championship
4,Adam Hadwin,1,46.0,66.36,2025-11-16,Port Royal Golf Course,Butterfield Bermuda Championship
5,Adam Long,2,28.5,25.94,2025-11-16,Port Royal Golf Course,Butterfield Bermuda Championship
6,Adam Schenk,2,27.5,25.03,2025-11-16,Port Royal Golf Course,Butterfield Bermuda Championship
7,Adam Scott,1,5.0,7.21,2025-11-16,Port Royal Golf Course,Butterfield Bermuda Championship
8,Adam Svensson,1,22.0,31.74,2025-11-16,Port Royal Golf Course,Butterfield Bermuda Championship
9,Adrian Meronk,1,90.0,129.84,2025-11-16,Port Royal Golf Course,Butterfield Bermuda Championship


## Merged Dataframe

In [21]:
importlib.reload(utils.db_utils)
from utils.db_utils import build_test_rows

this_week = build_test_rows(
    db_path=db_path,
    stats_df=stats_df,
    odds_df=odds_current,
    cuts_df=cuts_current,
    recent_form_df=recent_form_current,
    course_hist_df=course_hist_current,
    dk_df=dk,
    season=tournament_config["new"]["season"]
)

this_week.head()

,PLAYER,SGTTG_RANK,SGTTG,SGOTT_RANK,SGOTT,SGAPR_RANK,SGAPR,SGATG_RANK,SGATG,SGP_RANK,SGP,BIRDIES_RANK,BIRDIES,PAR_3_RANK,PAR_3,PAR_4_RANK,PAR_4,PAR_5_RANK,PAR_5,TOTAL_DRIVING_RANK,TOTAL_DRIVING,DRIVING_DISTANCE_RANK,DRIVING_DISTANCE,DRIVING_ACCURACY_RANK,DRIVING_ACCURACY,GIR_RANK,GIR,SCRAMBLING_RANK,SCRAMBLING,OWGR_RANK,OWGR,SEASON,VEGAS_ODDS,CUT_PERCENTAGE,FEDEX_CUP_POINTS,form_density,CONSECUTIVE_CUTS,RECENT_FORM,adj_form,COURSE_HISTORY,adj_ch,SALARY
0,Adam Hadwin,161.0,-0.671,133.0,-0.150,167.0,-0.476,107.0,-0.044,90.0,0.051,107.0,21.80%,50.0,3.03,129.0,4.04,142.0,4.65,58.0,162,96.0,302.8,66.0,62.14%,129.0,65.33%,160.0,55.17%,228.0,.6822,2025.0,125.0,50.0,110.038,5.50,1.0,70.8,23.25,46.0,66.36,6900
1,Adam Schenk,130.0,-0.155,92.0,0.066,115.0,-0.076,140.0,-0.145,110.0,-0.037,112.0,21.75%,36.0,3.02,142.0,4.05,86.0,4.60,132.0,203,59.0,307.1,144.0,56.26%,138.0,65.10%,121.0,58.07%,246.0,.6391,2025.0,125.0,36.8,173.425,9.13,4.0,71.9,24.00,27.5,25.03,7000
2,Adam Svensson,69.0,0.270,144.0,-0.235,58.0,0.230,24.0,0.274,171.0,-0.559,146.0,20.74%,68.0,3.04,119.0,4.03,49.0,4.57,89.0,177,153.0,297.0,24.0,66.54%,29.0,69.78%,98.0,59.07%,285.0,.5399,2025.0,90.0,44.4,131.550,7.31,2.0,69.7,23.67,22.0,31.74,6800
3,Alex Smalley,33.0,0.597,18.0,0.447,88.0,0.057,60.0,0.095,77.0,0.098,26.0,23.79%,108.0,3.07,6.0,3.96,44.0,4.56,9.0,92,51.0,308.4,41.0,64.58%,58.0,68.38%,24.0,63.29%,134.0,1.0870,2025.0,40.0,47.4,433.756,22.83,0.0,58.1,19.39,20.5,12.74,8700
4,Andrew Putnam,96.0,0.073,172.0,-0.716,36.0,0.347,8.0,0.443,25.0,0.360,134.0,21.02%,7.0,2.98,36.0,3.99,102.0,4.61,98.0,181,176.0,282.5,5.0,71.90%,14.0,70.80%,5.0,66.32%,173.0,.8558,2025.0,80.0,47.4,343.031,18.05,1.0,60.5,20.20,55.5,50.52,7100


### Dataframe Normalization

Run the same normalization techniques that I ran on the historical data:
- Any NaN Odds go to 1000/1 and clip all values at 1000/1
- OWGR NaN values go to 1000
- Recent form NaN values act like a MC at 90
- Average everything else.

In [22]:
# === 1. Clean percentage stats stored as strings like '62.5%' ===
percent_stats = ["SCRAMBLING", "DRIVING_ACCURACY", "BIRDIES", "GIR"]

for col in percent_stats:
    if col in this_week.columns:
        this_week[col] = (
            this_week[col]
            .astype(str)
            .str.replace('%', '', regex=False)
            .replace(['None', 'nan', 'NaN', '--', 'DNP', ''], np.nan)
        )
        this_week[col] = pd.to_numeric(this_week[col], errors="coerce")

# === 2. Normalize & assign fallback values for key fields ===
this_week["VEGAS_ODDS"] = pd.to_numeric(this_week["VEGAS_ODDS"], errors="coerce").fillna(1000).clip(upper=1000)

if "OWGR" in this_week.columns:
    this_week["OWGR"] = pd.to_numeric(this_week["OWGR"], errors="coerce")
    max_owgr = this_week["OWGR"].dropna().max()
    this_week["OWGR"] = this_week["OWGR"].fillna(max_owgr).clip(upper=1000)

if "OWGR_RANK" in this_week.columns:
    this_week["OWGR_RANK"] = pd.to_numeric(this_week["OWGR_RANK"], errors="coerce").fillna(1000).clip(upper=1000)

this_week["RECENT_FORM"] = pd.to_numeric(this_week["RECENT_FORM"], errors="coerce").fillna(90)
this_week["FEDEX_CUP_POINTS"] = pd.to_numeric(this_week["FEDEX_CUP_POINTS"], errors="coerce").fillna(0)

if "COURSE_HISTORY" in this_week.columns:
    this_week["COURSE_HISTORY"] = pd.to_numeric(this_week["COURSE_HISTORY"], errors="coerce")
    ch_mean = this_week["COURSE_HISTORY"].mean()
    this_week["COURSE_HISTORY"] = this_week["COURSE_HISTORY"].fillna(ch_mean)

# === 3. Fill all remaining NaNs in numeric columns ===
# Recast everything that might look numeric to be sure
for col in this_week.columns:
    if col not in ["PLAYER", "SALARY", "TOURNAMENT", "SEASON"]:
        try:
            this_week[col] = pd.to_numeric(this_week[col], errors="coerce")
        except Exception:
            continue

numeric_cols = this_week.select_dtypes(include=["number"]).columns.tolist()

for col in numeric_cols:
    if this_week[col].isna().any():
        col_mean = this_week[col].mean()
        this_week[col] = this_week[col].fillna(col_mean if not np.isnan(col_mean) else 0)


In [23]:
# === Final Check: No NaNs Should Remain ===
assert this_week.isna().sum().sum() == 0, "🚨 Still missing values in prediction set!"

In [24]:
# === Check for Remaining Missing Values ===
missing_summary = this_week.isna().sum()
missing_summary = missing_summary[missing_summary > 0].sort_values(ascending=False)

print("🧹 Columns still containing NaN values:")
display(missing_summary)

🧹 Columns still containing NaN values:


Series([], dtype: int64)

In [25]:
this_week.head(10)

,PLAYER,SGTTG_RANK,SGTTG,SGOTT_RANK,SGOTT,SGAPR_RANK,SGAPR,SGATG_RANK,SGATG,SGP_RANK,SGP,BIRDIES_RANK,BIRDIES,PAR_3_RANK,PAR_3,PAR_4_RANK,PAR_4,PAR_5_RANK,PAR_5,TOTAL_DRIVING_RANK,TOTAL_DRIVING,DRIVING_DISTANCE_RANK,DRIVING_DISTANCE,DRIVING_ACCURACY_RANK,DRIVING_ACCURACY,GIR_RANK,GIR,SCRAMBLING_RANK,SCRAMBLING,OWGR_RANK,OWGR,SEASON,VEGAS_ODDS,CUT_PERCENTAGE,FEDEX_CUP_POINTS,form_density,CONSECUTIVE_CUTS,RECENT_FORM,adj_form,COURSE_HISTORY,adj_ch,SALARY
0,Adam Hadwin,161.000000,-0.671000,133.000000,-0.150,167.00000,-0.476000,107.000000,-0.044000,90.000000,0.051000,107.000000,21.800000,50.000000,3.03,129.000000,4.04000,142.000000,4.650000,58.000000,162.000000,96.000000,302.800000,66.000000,62.140000,129.000000,65.330000,160.000000,55.17000,228.0,0.6822,2025.0,125.0,50.000000,110.038,5.500000,1.000000,70.8,23.25000,46.000000,66.360000,6900
1,Adam Schenk,130.000000,-0.155000,92.000000,0.066,115.00000,-0.076000,140.000000,-0.145000,110.000000,-0.037000,112.000000,21.750000,36.000000,3.02,142.000000,4.05000,86.000000,4.600000,132.000000,203.000000,59.000000,307.100000,144.000000,56.260000,138.000000,65.100000,121.000000,58.07000,246.0,0.6391,2025.0,125.0,36.800000,173.425,9.130000,4.000000,71.9,24.00000,27.500000,25.030000,7000
2,Adam Svensson,69.000000,0.270000,144.000000,-0.235,58.00000,0.230000,24.000000,0.274000,171.000000,-0.559000,146.000000,20.740000,68.000000,3.04,119.000000,4.03000,49.000000,4.570000,89.000000,177.000000,153.000000,297.000000,24.000000,66.540000,29.000000,69.780000,98.000000,59.07000,285.0,0.5399,2025.0,90.0,44.400000,131.550,7.310000,2.000000,69.7,23.67000,22.000000,31.740000,6800
3,Alex Smalley,33.000000,0.597000,18.000000,0.447,88.00000,0.057000,60.000000,0.095000,77.000000,0.098000,26.000000,23.790000,108.000000,3.07,6.000000,3.96000,44.000000,4.560000,9.000000,92.000000,51.000000,308.400000,41.000000,64.580000,58.000000,68.380000,24.000000,63.29000,134.0,1.0870,2025.0,40.0,47.400000,433.756,22.830000,0.000000,58.1,19.39000,20.500000,12.740000,8700
4,Andrew Putnam,96.000000,0.073000,172.000000,-0.716,36.00000,0.347000,8.000000,0.443000,25.000000,0.360000,134.000000,21.020000,7.000000,2.98,36.000000,3.99000,102.000000,4.610000,98.000000,181.000000,176.000000,282.500000,5.000000,71.900000,14.000000,70.800000,5.000000,66.32000,173.0,0.8558,2025.0,80.0,47.400000,343.031,18.050000,1.000000,60.5,20.20000,55.500000,50.520000,7100
5,Andrew Zielinski,106.829268,-0.090463,101.670732,-0.033,100.47561,-0.033585,98.756098,-0.023841,99.585366,-0.050195,96.158537,21.988171,80.073171,3.05,87.670732,4.01878,86.646341,4.598537,78.817073,168.695122,87.731707,303.929268,80.963415,61.719512,78.817073,67.596707,92.353659,59.37061,1000.0,2.2365,2025.0,1000.0,51.174747,0.000,16.236667,1.545455,90.0,23.70303,50.121667,48.479333,6000
6,Antoine Rozner,74.000000,0.197000,98.000000,0.038,19.00000,0.488000,167.000000,-0.329000,162.000000,-0.412000,68.000000,22.550000,2.000000,2.96,162.000000,4.08000,49.000000,4.570000,69.000000,168.000000,37.000000,310.500000,131.000000,58.030000,24.000000,69.980000,171.000000,53.31000,160.0,0.9033,2025.0,100.0,73.300000,177.046,11.800000,0.000000,60.9,21.97000,50.121667,48.479333,7100
7,Austin Cook,106.829268,-0.090463,101.670732,-0.033,100.47561,-0.033585,98.756098,-0.023841,99.585366,-0.050195,96.158537,21.988171,80.073171,3.05,87.670732,4.01878,86.646341,4.598537,78.817073,168.695122,87.731707,303.929268,80.963415,61.719512,78.817073,67.596707,92.353659,59.37061,665.0,0.1907,2025.0,500.0,50.000000,37.150,9.290000,1.000000,62.0,38.52000,59.300000,36.850000,6100
8,Beau Hossler,140.000000,-0.238000,151.000000,-0.301,159.00000,-0.386000,6.000000,0.449000,36.000000,0.261000,80.000000,22.350000,26.000000,3.01,119.000000,4.03000,62.000000,4.580000,158.000000,240.000000,81.000000,305.000000,159.000000,54.400000,155.000000,64.140000,60.000000,60.74000,121.0,1.2088,2025.0,45.0,75.000000,368.193,18.410000,1.000000,52.4,17.21000,46.700000,33.690000,7800
9,Ben Kohles,24.000000,0.741000,75.000000,0.117,9.0000

## Correlations
**Last 4 Years**

In [26]:
import plotly.express as px

# Get the 4 most recent seasons (or fewer if not available)
available_seasons = sorted(training_df["SEASON"].unique())[-4:]

# Dictionary to store correlation results
correlation_frames = {}

# Calculate correlations for each season
for season in available_seasons:
    df = training_df[training_df["SEASON"] == season].copy()

    # Numeric features only, drop final result
    numeric = df.select_dtypes(include=["float64", "int64"]).drop(columns=["FINAL_POS", "SEASON"], errors="ignore")

    # Skip if TOP_20 is not present
    if "TOP_20" not in numeric.columns:
        continue

    # Correlation with TOP_20
    cor = numeric.corr().abs()["TOP_20"].drop("TOP_20", errors="ignore").sort_values(ascending=False)
    cor_df = cor.reset_index()
    cor_df.columns = ["Feature", "Correlation"]
    correlation_frames[season] = cor_df

# Plot one bar chart per season
for season, cor_df in correlation_frames.items():
    fig = px.bar(
        cor_df,
        x="Feature",
        y="Correlation",
        title=f"🔎 Correlation to TOP_20 — Season {int(season)}",
        height=500,
        width=1000
    )

    fig.update_layout(
        xaxis_title="Feature",
        yaxis_title="Absolute Correlation to TOP_20",
        template="plotly_dark",
        xaxis_tickangle=-45
    )

    fig.show()


**Average of Last 4 Years**

In [27]:
import pandas as pd
import numpy as np
import plotly.express as px

# === Rebuild correlation matrices for the 4 most recent seasons ===
cor = {}
recent_seasons = sorted(training_df["SEASON"].unique())[-4:]

for season in recent_seasons:
    df_season = training_df[training_df["SEASON"] == season].copy()
    numeric_cols = df_season.select_dtypes(include=["float64", "int64"])
    cor[int(season)] = numeric_cols.corr().abs()

# === Combine into one summary DataFrame ===
cor_df = pd.DataFrame()

for season in recent_seasons:
    df = cor[season].reset_index()
    if "TOP_20" not in df.columns:
        continue
    temp = df[["index", "TOP_20"]].copy()
    temp = temp.rename(columns={"TOP_20": str(season)})
    if cor_df.empty:
        cor_df = temp
    else:
        cor_df = cor_df.merge(temp, on="index", how="outer")

# Drop target and metadata rows
cor_df = cor_df[~cor_df["index"].isin(["TOP_20", "FINAL_POS", "SEASON"])]

# Compute average correlation across seasons
cor_df["AVERAGE"] = cor_df.drop(columns=["index"]).mean(axis=1)
cor_df = cor_df.sort_values(by="AVERAGE", ascending=False)

# === Plotly Bar Chart ===
fig = px.bar(
    cor_df,
    x="index",
    y="AVERAGE",
    title="Average Correlation to TOP_20 (Last 4 Seasons)",
    labels={"index": "Feature", "AVERAGE": "Avg Correlation"},
    template="plotly_dark"
)

fig.update_layout(
    xaxis_tickangle=-45,
    height=600,
    width=1000,
    showlegend=False
)

fig.show()


# ML Model

## Predictors

In [28]:
# === Define Features and Target ===
target_col = "TOP_20"

# Columns to exclude from features
exclude = [
    "PLAYER", "TOURNAMENT", "COURSE", "ENDING_DATE", "SEASON", "TOURN_ID",
    "TOP_20", "FINAL_POS"  
]

# Select numeric feature columns
feature_cols = [
    col for col in training_df.columns
    if col not in exclude and training_df[col].dtype in [np.float64, np.int64]
]

print(f"Selected {len(feature_cols)} features:\n", feature_cols)

Selected 39 features:
 ['SGTTG_RANK', 'SGTTG', 'SGOTT_RANK', 'SGOTT', 'SGAPR_RANK', 'SGAPR', 'SGATG_RANK', 'SGATG', 'SGP_RANK', 'SGP', 'BIRDIES_RANK', 'BIRDIES', 'PAR_3_RANK', 'PAR_3', 'PAR_4_RANK', 'PAR_4', 'PAR_5_RANK', 'PAR_5', 'TOTAL_DRIVING_RANK', 'TOTAL_DRIVING', 'DRIVING_DISTANCE_RANK', 'DRIVING_DISTANCE', 'DRIVING_ACCURACY_RANK', 'DRIVING_ACCURACY', 'GIR_RANK', 'GIR', 'SCRAMBLING_RANK', 'SCRAMBLING', 'OWGR_RANK', 'OWGR', 'VEGAS_ODDS', 'CUT_PERCENTAGE', 'FEDEX_CUP_POINTS', 'form_density', 'CONSECUTIVE_CUTS', 'RECENT_FORM', 'adj_form', 'COURSE_HISTORY', 'adj_ch']


### Random Forest
Initial default model before hyperparameter tuning.

This one actually performed really well (ROC AUC above 0.85).  Strangely doing hyperparameter tuning hurt the model (less than 0.7), so I removed that step and we will just use this default model.  This does cross-validation testing on 5 folds to ensure robust train/test splits.

In [29]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline

# === Standardize features ===
scaler = StandardScaler()
X_rf_std = scaler.fit_transform(training_df[feature_cols])
y_rf = training_df["TOP_20"]

# === Balance the dataset with SMOTE and undersampling ===
over = SMOTE(sampling_strategy=0.5, k_neighbors=3)
under = RandomUnderSampler(sampling_strategy=0.5)
pipeline = Pipeline([("o", over), ("u", under)])
X_rf_bal, y_rf_bal = pipeline.fit_resample(X_rf_std, y_rf)

# === Define and evaluate Random Forest ===
rf = RandomForestClassifier(n_estimators=100, random_state=42)
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scores = cross_val_score(rf, X_rf_bal, y_rf_bal, cv=cv, scoring="roc_auc")

print("📊 Random Forest (All Features)")
print("Cross-validation scores:", scores)
print("Average ROC AUC score: {:.2f}".format(scores.mean()))


📊 Random Forest (All Features)
Cross-validation scores: [0.90512821 0.921337   0.9017598  0.91151997 0.91327663]
Average ROC AUC score: 0.91


Random Forest Training & Feature Importance

Now we train on all the data (no longer cross-validation folds) and extract the feature importance to see how it compares with linear correlations.


In [30]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.preprocessing import StandardScaler
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
import pandas as pd
import plotly.express as px

# === Resample the full dataset ===
over = SMOTE(sampling_strategy=0.5, k_neighbors=3)
under = RandomUnderSampler(sampling_strategy=0.5)
resample_pipeline = Pipeline([("o", over), ("u", under)])
X_all_resampled, y_all_resampled = resample_pipeline.fit_resample(
    training_df[feature_cols], training_df["TOP_20"]
)

# === Standardize ===
scaler = StandardScaler()
X_all_scaled = scaler.fit_transform(X_all_resampled)

# === Train Calibrated Random Forest ===
base_rf = RandomForestClassifier(random_state=42)
final_model = CalibratedClassifierCV(estimator=base_rf, method="sigmoid", cv=5)
final_model.fit(X_all_scaled, y_all_resampled)

# === Aggregate feature importances across all CV folds ===
all_importances = np.array([
    est.estimator.feature_importances_ for est in final_model.calibrated_classifiers_
])
mean_importance = all_importances.mean(axis=0)

importance_df = pd.DataFrame({
    "Feature": feature_cols,
    "Importance": mean_importance
}).sort_values(by="Importance", ascending=False)

# === Plot feature importances ===
fig = px.bar(
    importance_df,
    x="Feature",
    y="Importance",
    title="Random Forest Feature Importances",
    template="plotly_dark"
)
fig.update_layout(xaxis_tickangle=-45)
fig.show()


Use this model on this week's data to predict the best performers.

In [31]:
# === Prepare test data (this week's players) ===

# Select and standardize features using the same scaler as training
X_test = this_week[feature_cols]
X_test_scaled = scaler.transform(X_test)

# === Predict probabilities for Top 20 finish ===
# CalibratedClassifierCV returns well-calibrated probabilities
this_week["PROBABILITY"] = final_model.predict_proba(X_test_scaled)[:, 1]

# === Sort results by highest predicted probability ===
this_week_sorted = this_week.sort_values(by="PROBABILITY", ascending=False).reset_index(drop=True)

# === Define final column order explicitly ===
columns_to_show = [
    "PLAYER", "SALARY", "PROBABILITY",
    "RECENT_FORM", "COURSE_HISTORY", "VEGAS_ODDS", "CUT_PERCENTAGE", 
    "OWGR_RANK", "FEDEX_CUP_POINTS",

    # SG stats
    "SGTTG_RANK", "SGOTT_RANK", "SGAPR_RANK", "SGATG_RANK", "SGP_RANK",

    # Scoring/Performance
    "BIRDIES_RANK", "PAR_3_RANK", "PAR_4_RANK", "PAR_5_RANK",

    # Driving / Ball Striking
    "TOTAL_DRIVING_RANK", "DRIVING_DISTANCE_RANK", "DRIVING_ACCURACY_RANK",
    "GIR_RANK", "SCRAMBLING_RANK"
]

# Keep only columns that exist in the dataframe
columns_to_show = [col for col in columns_to_show if col in this_week_sorted.columns]

# Final dataframe for display/export
export_df = this_week_sorted[columns_to_show].copy()

# Preview top 20
export_df.head(20)


,PLAYER,SALARY,PROBABILITY,RECENT_FORM,COURSE_HISTORY,VEGAS_ODDS,CUT_PERCENTAGE,OWGR_RANK,FEDEX_CUP_POINTS,SGTTG_RANK,SGOTT_RANK,SGAPR_RANK,SGATG_RANK,SGP_RANK,BIRDIES_RANK,PAR_3_RANK,PAR_4_RANK,PAR_5_RANK,TOTAL_DRIVING_RANK,DRIVING_DISTANCE_RANK,DRIVING_ACCURACY_RANK,GIR_RANK,SCRAMBLING_RANK
0,Patrick Rodgers,9500,0.905195,52.5,33.000000,30.0,62.5,118.0,811.269,94.000000,88.000000,116.00000,63.000000,92.000000,54.000000,50.000000,36.000000,102.000000,40.000000,45.000000,105.000000,117.000000,33.000000
1,Beau Hossler,7800,0.858674,52.4,46.700000,45.0,75.0,121.0,368.193,140.000000,151.000000,159.00000,6.000000,36.000000,80.000000,26.000000,119.000000,62.000000,158.000000,81.000000,159.000000,155.000000,60.000000
2,Jeremy Paul,6900,0.804820,62.0,50.121667,90.0,50.0,271.0,203.663,140.000000,118.000000,141.00000,89.000000,94.000000,93.000000,26.000000,36.000000,69.000000,81.000000,51.000000,123.000000,43.000000,41.000000
3,Greyson Sigg,7400,0.768436,65.9,14.000000,50.0,41.2,258.0,200.896,35.000000,93.000000,27.00000,62.000000,167.000000,149.000000,68.000000,24.000000,127.000000,77.000000,148.000000,25.000000,27.000000,34.000000
4,Quade Cummins,7000,0.733695,63.9,50.121667,125.0,64.7,266.0,132.191,149.000000,102.000000,150.00000,142.000000,53.000000,126.000000,68.000000,78.000000,117.000000,121.000000,61.000000,134.000000,107.000000,36.000000
5,Mark Hubbard,8500,0.706442,55.8,35.200000,45.0,60.0,113.0,419.879,103.000000,115.000000,81.00000,111.000000,48.000000,66.000000,68.000000,57.000000,69.000000,74.000000,105.000000,67.000000,81.000000,76.000000
6,Cameron Champ,7300,0.665009,46.1,90.000000,80.0,66.7,291.0,224.955,106.829268,101.670732,100.47561,98.756098,99.585366,96.158537,80.073171,87.670732,86.646341,78.817073,87.731707,80.963415,78.817073,92.353659
7,Zac Blair,6700,0.664529,63.3,73.500000,125.0,58.3,336.0,92.229,88.000000,154.000000,26.00000,82.000000,137.000000,82.000000,36.000000,99.000000,86.000000,89.000000,167.000000,10.000000,77.000000,90.000000
8,Michael Brennan,9300,0.643212,30.0,50.121667,30.0,100.0,40.0,505.000,106.829268,101.670732,100.47561,98.756098,99.585366,96.158537,80.073171,87.670732,86.646341,78.817073,87.731707,80.963415,78.817073,92.353659
9,Sam Ryder,7500,0.639341,54.3,61.700000,70.0,68.4,197.0,314.543,135.000000,132.000000,101.00000,114.000000,10.000000,53.000000,18.000000,24.000000,166.000000,121.000000,108.000000,87.000000,114.000000,26.000000


### Save to CSV

In [32]:
# Ensure required columns for spreadsheet export
if "COURSE_HISTORY" not in export_df.columns:
    export_df["COURSE_HISTORY"] = "-"

# Define columns to round (FedEx + stats)
columns_to_round = [
    "FEDEX_CUP_POINTS",
    "SGTTG_RANK", "SGOTT_RANK", "SGAPR_RANK", "SGATG_RANK", "SGP_RANK",
    "BIRDIES_RANK", "PAR_3_RANK", "PAR_4_RANK", "PAR_5_RANK",
    "TOTAL_DRIVING_RANK", "DRIVING_DISTANCE_RANK", "DRIVING_ACCURACY_RANK",
    "GIR_RANK", "SCRAMBLING_RANK"
]

columns_to_round_1 = [
    "COURSE_HISTORY", "CUT_PERCENTAGE"
]

# Round only the selected columns to 0 decimal places
export_df[columns_to_round] = export_df[columns_to_round].round(0).astype("Int64")
# Round only the selected columns to 1 decimal places
export_df[columns_to_round_1] = export_df[columns_to_round_1].round(1)

# Explicit column order for Excel integration
column_order = [
    "PLAYER", "SALARY", "PROBABILITY", "RECENT_FORM", "COURSE_HISTORY", "VEGAS_ODDS", 
    "CUT_PERCENTAGE", "OWGR_RANK", "FEDEX_CUP_POINTS", 
    "SGTTG_RANK", "SGOTT_RANK", "SGAPR_RANK", "SGATG_RANK", "SGP_RANK",
    "BIRDIES_RANK", "PAR_3_RANK", "PAR_4_RANK", "PAR_5_RANK",
    "TOTAL_DRIVING_RANK", "DRIVING_DISTANCE_RANK", "DRIVING_ACCURACY_RANK",
    "GIR_RANK", "SCRAMBLING_RANK"
]

# Ensure all expected columns exist
for col in column_order:
    if col not in export_df.columns:
        export_df[col] = "-" if col == "COURSE_HISTORY" else np.nan

# Reorder columns to match Excel layout
export_df = export_df[column_order]

# Export to CSV
filename = f"data/current_week_export.csv"
export_df.to_csv(filename, index=False)
print(f"✅ Exported to {filename}")

✅ Exported to data/current_week_export.csv
